In [1]:
import numpy as np
from lensing_data_class import LensingData
from cluster_local_tidy import ClusterLensing
import os
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM
from cluster_local_new import ClusterLensing_fyp
import pandas as pd
import corner
import arviz as az
import pathlib

In [2]:
import warnings

# Suppressing the lenstronomy warning on astropy.cosmology
from lenstronomy.LensModel.lens_model import LensModel
warnings.filterwarnings("ignore", category=UserWarning, module='lenstronomy.LensModel.lens_model')

In [3]:


# --- 1. Create Dummy Lensing Data ---
# In a real scenario, you would load your FITS files or other data here.
# We'll create placeholder data for 2 clusters.
# The maps are small for this example (e.g., 200x200 pixels).

scenarios = {
        '1': 'abell370',
        '2': 'abell2744',
        '3': 'abells1063',
        '4': 'macs0416',
        '5': 'macs0717',
        '6': 'macs1149'
    }

full_cluster_names = {
        'abell370': 'Abell 370',
        'abell2744': 'Abell 2744',
        'abells1063': 'Abell S1063',
        'macs0416': 'MACS J0416.1-2403',
        'macs0717': 'MACS J0717.5+3745',
        'macs1149': 'MACS J1149.5+2223'
    }

# Initialize lists to store the data arrays
datax_list = []
datay_list = []
data_psi_list = []

for i in scenarios:
    clustername = scenarios[i]
    full_cluster_name = full_cluster_names[clustername]

    file_dir = os.getcwd()
    fits_filex = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_x-arcsec-deflect.fits'
    )
    fits_filey = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_y-arcsec-deflect.fits'
    )
    psi_file = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_psi.fits'
    )

    with fits.open(fits_filex) as hdulx, fits.open(fits_filey) as hduly, fits.open(psi_file) as hdul_psi:
        datax = hdulx[0].data
        datay = hduly[0].data
        data_psi = hdul_psi[0].data

        # Append the data arrays to the lists
        datax_list.append(datax)
        datay_list.append(datay)
        data_psi_list.append(data_psi)

# getting the pixel scale list
pixscale_list = [0.2, 0.25, 0.25, 0.2, 0.2, 0.2]
lensing_data = LensingData(
    alpha_maps_x=datax_list,
    alpha_maps_y=datay_list,
    lens_potential_maps=data_psi_list,
    pixscale = [0.2, 0.25, 0.25, 0.2, 0.2, 0.2],
    z_l_list = [0.375, 0.308, 0.351, 0.397, 0.545, 0.543], # Lens redshifts for the two clusters
    # We can use the default x_center, y_center, and search_window_list
    # or override them if needed.
)

# --- 2. Define the "True" Observed Data ---
# This is the data you are trying to fit. For a strongly lensed supernova,
# this would be the measured time delays between the images.
# Let's assume a 4-image system.
dt_true = np.array([   0.        , 4722.09151606, 5726.87137736, 6178.592956  ,
       6376.37992613, 6552.76725006]) # First image is reference (delay=0)

# --- 3. Initialize the Main Analysis Class ---
# z_s_ref is a reference source redshift used for initial scaling calculations.
# It can be an estimate of the source's redshift.
z_s_ref = 1.5 
cluster_system = ClusterLensing(data=lensing_data, z_s_ref=z_s_ref)

print("Setup complete. Lensing system initialized.")

Setup complete. Lensing system initialized.


In [4]:
# test image pos and dt
test_params = {"x_src" : 80.5, "y_src": 74.0, "z_s": 3.2, "H0": 70.0}
test_cluster = 0
# Calculate the image positions and time delays for the test parameters
output = cluster_system.calculate_images_and_delays(
    test_params, test_cluster
)
print(output['time_delays'])


[   0.         4722.09151606 5726.87137736 6178.592956   6376.37992613
 6552.76725006]


In [5]:
# test chi sq calculation
test_params_chi_sq = {
    "x_src": 80.5, 
    "y_src": 74.0, 
    "z_s": 3.2, 
    "H0": 69.7
}
chi_sq_value = cluster_system._calculate_chi_squared(
    params=test_params_chi_sq,
    dt_true=dt_true,
    index=test_cluster,
    sigma_dt=0.05 # Assumed fractional error on time delays
)

print(f"Chi-Square value for the test parameters: {chi_sq_value}")

# Example with a bad parameter set that produces a different number of images
bad_params = {"x_src": 9.0, "y_src": 10.0} # Likely to produce 0 or 1 image
chi_sq_penalty = cluster_system._calculate_chi_squared(
    params=bad_params, dt_true=dt_true, index=test_cluster
)
print(f"Chi-Square penalty for a bad parameter set: {chi_sq_penalty}")

Chi-Square value for the test parameters: 0.03705159846429336
Chi-Square penalty for a bad parameter set: 7348.469228349533


In [ ]:
# test only de
# --- Running the DE Optimization ---
# The goal is to find which cluster model (0 or 1) and which source position
# best reproduce the observed time delays dt_true.

print("\n--- Running Differential Evolution (DE) to find the best fit ---")

# Optional: DE settings can be customized. Using defaults here.
# de_settings = {'popsize': 50, 'maxiter': 300, 'disp': False}
best_result_de = cluster_system.find_best_fit(
    dt_true=dt_true,
    run_mcmc=False, # We are NOT running MCMC yet
    # de_settings=de_settings 
)

print("\n--- DE Optimization Complete ---")
print(f"Best cluster index: {best_result_de['cluster_index']}")
print(f"Minimum Chi-Square: {best_result_de['chi_sq']:.3f}")
print("Best-fit parameters:")
for param, value in best_result_de['params'].items():
    print(f"  {param}: {value:.4f}")

In [ ]:
# test de then mcmc 
print("\nRunning the full analysis pipeline...")

cosmos = FlatLambdaCDM(H0=test_params['H0'], Om0=0.3)
lum_dist_true = cosmos.luminosity_distance(test_params['z_s']).value  # Luminosity distance in Mpc
print("True luminosity distances:", lum_dist_true)

# Define settings for the MCMC sampler
mcmc_settings = {
    "n_walkers": 20,      # Number of MCMC walkers
    "n_steps": 8000,      # Number of steps per walker
    "fit_z": True,        # We want to fit for the source redshift (z_s)
    "fit_hubble": True,   # We want to fit for the Hubble constant (H0)
    "lum_dist_true": lum_dist_true, # An external "true" luminosity distance measurement (in Mpc)
    "sigma_lum": 0.05,     # The fractional error on the luminosity distance
    # Define the prior boundaries for the parameters being fit in the MCMC
    "z_bounds": (1.0, 5.0),
    "H0_bounds": (60, 90)
}

# Call the main analysis function
# The function will first run DE to find the best cluster,
# then run MCMC on that best-fit model.
mcmc_results, accepted_cluster_indices = cluster_system.find_best_fit(
    dt_true=dt_true,
    run_mcmc=True,
    mcmc_settings=mcmc_settings
)
print(f"\nAnalysis complete. Found {len(mcmc_results)} cluster(s) meeting the MCMC criterion.")


In [ ]:
OUT_DIR      = pathlib.Path("/home/dices/Research/GWlensing_Brian/oddratio/test_tidy_1")
OUT_DIR.mkdir(exist_ok=True)

for result in mcmc_results:
        cluster_idx = result['cluster_index']
        print(f"\n--- Processing Final Results for Cluster {cluster_idx} ---")
        
        # --- Step 5: Analyze and Display ---
        sampler = result['mcmc_sampler']
        burn_in_steps = 800
        labels = list(result['params'].keys())
        flat_samples = sampler.get_chain(discard=burn_in_steps, thin=15, flat=True)
        
        print(f"--- MCMC Parameter Constraints for Cluster {cluster_idx} ---")
        for i in range(len(labels)):
            mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
            q = np.diff(mcmc)
            print(f"{labels[i]:>7s} = {mcmc[1]:.3f} +{q[1]:.3f} / -{q[0]:.3f}")

        # --- Step 6: Save the Localization Results ---
        print(f"Saving localization results for Cluster {cluster_idx}...")
        file_path = os.path.join(OUT_DIR, f"cluster_{cluster_idx}_posterior.npz")
        
        cluster_system.save_mcmc_results(
            sampler=sampler,
            best_result=result, # Pass the individual result dictionary
            n_burn_in=burn_in_steps,
            output_path=file_path,
            dt_true=dt_true,
            mcmc_settings=mcmc_settings
        )

print("\nAll processing finished.")